In [ ]:
'''
    remote_exec
    IBM/Dell Qiskit Runtime Emulator Demo
    July 26, 2021
    
    
    The following notebook walks through a (simple) example usage of the 
    Qiskit Runtime Emulator in a remote execution environment: i.e. one in
    a Kubernetes cluster or deployed anywhere that is not the client machine.
'''


from qiskit_emulator import EmulatorProvider
from qiskit import QuantumCircuit
import logging
import requests
import time
import os

RUNTIME_PROGRAM = """
# This code is part of qiskit-runtime.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):
    circuits = transpile(
        circuits,
    )
    
    user_messenger.publish({'results': 'intermittently'}, final=False)

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results
    result = backend.run(circuits, **kwargs).result()

    user_messenger.publish(result.to_dict(), final=True)
"""

RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

PROGRAM_PREFIX = 'qiskit-test'
REMOTE_RUNTIME = os.getenv("SERVER_URL") 

logging.basicConfig(level=logging.DEBUG)

In [16]:
'''
    The main difference between the local and remote runtime interfaces
    is here: `provider.remote(URL_OF_REMOTE)`
    
    This function connects the client to the remote environment and, if 
    necessary, logs them in through SSO if enabled. 
'''

provider = EmulatorProvider()
provider.remote(REMOTE_RUNTIME)

DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/status
DEBUG:urllib3.connectionpool:Resetting dropped connection: qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /status HTTP/1.1" 200 5
DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/status RESPONSE: (200, 'OK', 'false')
DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/sso_enabled
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /sso_enabled HTTP/1.1" 200 4
DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/sso_enabled RESPONSE: (200, 'OK', 'true')
DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/authenticate: {'token': 'eyJhbGciOiJSUzI1NiI

In [17]:
'''
    Uploading a program works exactly the same as locally
'''

text_program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)
print(f"PROGRAM ID: {text_program_id}")

DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program: {'data': "\n# This code is part of qiskit-runtime.\n#\n# (C) Copyright IBM 2021.\n#\n# This code is licensed under the Apache License, Version 2.0. You may\n# obtain a copy of this license in the LICENSE.txt file in the root directory\n# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.\n#\n# Any modifications or derivative works of this code must retain this\n# copyright notice, and modified files need to carry a notice indicating\n# that they have been altered from the originals.\nfrom qiskit.compiler import transpile, schedule\n\n\ndef main(\n    backend,\n    user_messenger,\n    circuits,\n    **kwargs,\n):\n    circuits = transpile(\n        circuits,\n    )\n    \n    user_messenger.publish({'results': 'intermittently'}, final=False)\n\n    if not isinstance(circuits, list):\n        circuits = [circuits]\n\n    # Compute raw results\n    result = 

PROGRAM ID: 998e4c1f2f3a


In [18]:
programs = provider.runtime.pprint_programs(refresh=True)

DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /program HTTP/1.1" 200 9330
DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program RESPONSE: (200, 'OK', '[{"program_id": "7883b5bdfa34", "name": "7883b5bdfa34", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"program_id": "7192bd40cf3a", "name": "7192bd40cf3a", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"program_id": "9f144321ca62", "name": "9f144321ca62", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"program_id": "a2d35ba27dde", "name": "a2d35ba27dde", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"progr

7883b5bdfa34:
  Name: 7883b5bdfa34
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
7192bd40cf3a:
  Name: 7192bd40cf3a
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
9f144321ca62:
  Name: 9f144321ca62
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
a2d35ba27dde:
  Name: a2d35ba27dde
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
8b35723d70b8:
  Name: 8b35723d70b8
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none


In [19]:
provider.runtime.update_program(text_program_id, description="IBM/Dell Updated Qiskit Runtime Program")

DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program/998e4c1f2f3a/update: {'program_id': '998e4c1f2f3a', 'data': None, 'name': None, 'data_type': None, 'program_metadata': '{"description": "IBM/Dell Updated Qiskit Runtime Program"}'}
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "POST /program/998e4c1f2f3a/update HTTP/1.1" 200 0
DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program/998e4c1f2f3a/update RESPONSE: (200, 'OK', '')
DEBUG:qiskit_emulator.remote_runtime_service:Successfully updated program 998e4c1f2f3a


True

In [20]:
programs = provider.runtime.pprint_programs(refresh=True)

DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /program HTTP/1.1" 200 9350
DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program RESPONSE: (200, 'OK', '[{"program_id": "7883b5bdfa34", "name": "7883b5bdfa34", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"program_id": "7192bd40cf3a", "name": "7192bd40cf3a", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"program_id": "9f144321ca62", "name": "9f144321ca62", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"program_id": "a2d35ba27dde", "name": "a2d35ba27dde", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"progr

7883b5bdfa34:
  Name: 7883b5bdfa34
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
7192bd40cf3a:
  Name: 7192bd40cf3a
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
9f144321ca62:
  Name: 9f144321ca62
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
a2d35ba27dde:
  Name: a2d35ba27dde
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
8b35723d70b8:
  Name: 8b35723d70b8
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none


In [21]:
'''
    This can also be done locally - but if the provided string to upload_program
    is a file or directory path, the upload function will grab the contents 
    and send those to the remote orchestrator.
'''


file_program_id = provider.runtime.upload_program("qka.py", description="File Upload to Orchestrator")

DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program: {'data': None, 'name': None, 'data_type': 'STRING', 'program_metadata': '{"description": "File Upload to Orchestrator"}'}
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "POST /program HTTP/1.1" 200 12
DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program RESPONSE: (200, 'OK', '15858c2c88c4')
DEBUG:qiskit_emulator.remote_runtime_service:Received program_id: 15858c2c88c4


In [22]:
dir_program_id = provider.runtime.upload_program("./qkad", description="Directory Upload to Orchestrator")

DEBUG:qiskit_emulator.remote_runtime_service:Have directory: ./qkad
DEBUG:qiskit_emulator.remote_runtime_service:made: /home/geoff/workspace/qiskit-runtime-emulator/examples/demo_jul26/qkad.zip
DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program: {'data': None, 'name': None, 'data_type': 'DIR', 'program_metadata': '{"description": "Directory Upload to Orchestrator"}'}
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "POST /program HTTP/1.1" 200 12
DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program RESPONSE: (200, 'OK', '634938bb2277')
DEBUG:qiskit_emulator.remote_runtime_service:Received program_id: 634938bb2277


In [23]:
provider.runtime.pprint_programs(refresh=True)

DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /program HTTP/1.1" 200 9615
DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program RESPONSE: (200, 'OK', '[{"program_id": "7883b5bdfa34", "name": "7883b5bdfa34", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"program_id": "7192bd40cf3a", "name": "7192bd40cf3a", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"program_id": "9f144321ca62", "name": "9f144321ca62", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"program_id": "a2d35ba27dde", "name": "a2d35ba27dde", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}, {"progr

7883b5bdfa34:
  Name: 7883b5bdfa34
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
7192bd40cf3a:
  Name: 7192bd40cf3a
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
9f144321ca62:
  Name: 9f144321ca62
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
a2d35ba27dde:
  Name: a2d35ba27dde
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
8b35723d70b8:
  Name: 8b35723d70b8
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none


In [24]:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

In [25]:
job = provider.runtime.run(text_program_id, options=None, inputs=program_inputs)

DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program/998e4c1f2f3a/job: {"circuits": {"__type__": "dill", "__value__": "gASVSQYAAAAAAACMHXFpc2tpdC5jaXJjdWl0LnF1YW50dW1jaXJjdWl0lIwOUXVhbnR1bUNpcmN1aXSUk5QpgZR9lCiMCl9iYXNlX25hbWWUjAdjaXJjdWl0lIwEbmFtZZSMCWNpcmN1aXQtM5SMBV9kYXRhlF2UKIwncWlza2l0LmNpcmN1aXQubGlicmFyeS5zdGFuZGFyZF9nYXRlcy5olIwFSEdhdGWUk5QpgZR9lCiMBl9sYWJlbJROjAtfZGVmaW5pdGlvbpROaAeMAWiUjApudW1fcXViaXRzlEsBjApudW1fY2xiaXRzlEsAjAdfcGFyYW1zlF2UjAljb25kaXRpb26UTowJX2R1cmF0aW9ulE6MBV91bml0lIwCZHSUdWJdlIwecWlza2l0LmNpcmN1aXQucXVhbnR1bXJlZ2lzdGVylIwFUXViaXSUk5QpgZROfZQojAZfaW5kZXiUSwCMCV9yZWdpc3RlcpRoHIwPUXVhbnR1bVJlZ2lzdGVylJOUKYGUTn2UKIwFX25hbWWUjAFxlIwFX3NpemWUSwKMBV9iaXRzlF2UKGgfaB4pgZROfZQoaCFLAWgiaCWMBV9oYXNolIoIYMkLA2R5CiGMBV9yZXBylIwhUXViaXQoUXVhbnR1bVJlZ2lzdGVyKDIsICdxJyksIDEplHWGlGJlaC6KCP//Us6hgZLvaC+MF1F1YW50dW1SZWdpc3RlcigyLCAncScplHWGlGJoLooIB2K7GZqwusZoL4whUXViaXQoUXVhbnR1bVJlZ2lzdGVyKDIsICdxJyksIDAplHWGlGJhXZ

In [26]:
results = job.result(timeout=60)
print(results)

{'backend_name': 'aer_simulator', 'backend_version': '0.8.2', 'qobj_id': '5079dfc3-181b-4123-94ff-61a016c8eb42', 'job_id': '24d8502a-cdf4-45ca-b125-47990a9ceed1', 'success': True, 'results': [{'shots': 1024, 'success': True, 'data': {'counts': {'0x0': 515, '0x3': 509}}, 'meas_level': 2, 'header': {'clbit_labels': [['c', 0], ['c', 1]], 'creg_sizes': [['c', 2]], 'global_phase': 0.0, 'memory_slots': 2, 'metadata': None, 'n_qubits': 2, 'name': 'circuit-3', 'qreg_sizes': [['q', 2]], 'qubit_labels': [['q', 0], ['q', 1]]}, 'status': 'DONE', 'seed_simulator': 2381170942, 'metadata': {'parallel_state_update': 4, 'measure_sampling': True, 'method': 'stabilizer', 'parallel_shots': 1, 'device': 'CPU', 'fusion': {'enabled': False}}, 'time_taken': 0.005178820000000001}], 'date': '2021-07-27T20:28:09.422807', 'status': 'COMPLETED', 'header': {'backend_name': 'aer_simulator', 'backend_version': '0.8.2'}, 'metadata': {'time_taken': 0.0055541570000000005, 'parallel_experiments': 1, 'omp_enabled': True, 

In [27]:
def callback_function(msg):
    print(f'******************\n\n\nFrom Callback Function: {msg}\n\n\n******************')
    

In [28]:
program_inputs['backend_name'] = 'emulator'
job = provider.runtime.run(dir_program_id, inputs=program_inputs, options=None, callback=callback_function)

DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program/634938bb2277/job: {"circuits": {"__type__": "dill", "__value__": "gASVSQYAAAAAAACMHXFpc2tpdC5jaXJjdWl0LnF1YW50dW1jaXJjdWl0lIwOUXVhbnR1bUNpcmN1aXSUk5QpgZR9lCiMCl9iYXNlX25hbWWUjAdjaXJjdWl0lIwEbmFtZZSMCWNpcmN1aXQtM5SMBV9kYXRhlF2UKIwncWlza2l0LmNpcmN1aXQubGlicmFyeS5zdGFuZGFyZF9nYXRlcy5olIwFSEdhdGWUk5QpgZR9lCiMBl9sYWJlbJROjAtfZGVmaW5pdGlvbpROaAeMAWiUjApudW1fcXViaXRzlEsBjApudW1fY2xiaXRzlEsAjAdfcGFyYW1zlF2UjAljb25kaXRpb26UTowJX2R1cmF0aW9ulE6MBV91bml0lIwCZHSUdWJdlIwecWlza2l0LmNpcmN1aXQucXVhbnR1bXJlZ2lzdGVylIwFUXViaXSUk5QpgZROfZQojAZfaW5kZXiUSwCMCV9yZWdpc3RlcpRoHIwPUXVhbnR1bVJlZ2lzdGVylJOUKYGUTn2UKIwFX25hbWWUjAFxlIwFX3NpemWUSwKMBV9iaXRzlF2UKGgfaB4pgZROfZQoaCFLAWgiaCWMBV9oYXNolIoIYMkLA2R5CiGMBV9yZXBylIwhUXViaXQoUXVhbnR1bVJlZ2lzdGVyKDIsICdxJyksIDEplHWGlGJlaC6KCP//Us6hgZLvaC+MF1F1YW50dW1SZWdpc3RlcigyLCAncScplHWGlGJoLooIB2K7GZqwusZoL4whUXViaXQoUXVhbnR1bVJlZ2lzdGVyKDIsICdxJyksIDAplHWGlGJhXZ

DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/83ebc22669be/status HTTP/1.1" 200 7
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/83ebc22669be/results
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/83ebc22669be/results HTTP/1.1" 200 16
DEBUG:qiskit_emulator.emulator_runtime_job:got: {'messages': []}
DEBUG:qiskit_emulator.emulator_runtime_job:result: got []
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/83ebc22669be/status
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/83ebc22669be/status HTTP/1.1" 200 7
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/83ebc22669be/results
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/83ebc22669be/resul

******************


From Callback Function: {'cost': 11.842808462349002, 'kernel_parameters': {'__type__': 'array', '__value__': [0.24756947922942354]}}


******************


DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/83ebc22669be/status HTTP/1.1" 200 7
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/83ebc22669be/results
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/83ebc22669be/results/2021-07-27 20:29:20
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/83ebc22669be/results/2021-07-27%2020:29:20 HTTP/1.1" 200 16
DEBUG:qiskit_emulator.emulator_runtime_job:got: {'messages': []}
DEBUG:qiskit_emulator.emulator_runtime_job:result: got []
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/83ebc22669be/status
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/83ebc22669be/status HTTP/1.1" 200 7
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-sm

DEBUG:qiskit_emulator.emulator_runtime_job:result: got [{'data': '{"final": true, "message": {"aligned_kernel_parameters": {"__type__": "array", "__value__": [0.24756947922942354]}, "aligned_kernel_matrix": {"__type__": "array", "__value__": [[1.0, 0.0029296875, 0.0042724609375, 0.048583984375, 0.0523681640625, 0.0, 0.0001220703125, 0.0029296875, 0.005615234375, 0.050048828125, 0.0047607421875, 0.02392578125, 0.0543212890625, 0.01123046875, 0.0025634765625, 0.0177001953125, 0.6767578125, 0.0042724609375, 0.2142333984375, 0.0501708984375], [0.0029296875, 1.0, 0.0, 0.0003662109375, 0.0003662109375, 0.0003662109375, 0.07177734375, 0.00830078125, 0.004150390625, 0.0628662109375, 0.0224609375, 0.00146484375, 0.001220703125, 0.3709716796875, 0.031005859375, 0.00732421875, 0.0059814453125, 0.026611328125, 0.02783203125, 0.080810546875], [0.0042724609375, 0.0, 1.0, 0.068603515625, 0.059814453125, 0.0474853515625, 0.0001220703125, 0.0040283203125, 0.00244140625, 0.0003662109375, 0.0, 0.00036621

In [29]:
results = job.result(timeout=600)
print(results)

{'aligned_kernel_parameters': {'__type__': 'array', '__value__': [0.24756947922942354]}, 'aligned_kernel_matrix': {'__type__': 'array', '__value__': [[1.0, 0.0029296875, 0.0042724609375, 0.048583984375, 0.0523681640625, 0.0, 0.0001220703125, 0.0029296875, 0.005615234375, 0.050048828125, 0.0047607421875, 0.02392578125, 0.0543212890625, 0.01123046875, 0.0025634765625, 0.0177001953125, 0.6767578125, 0.0042724609375, 0.2142333984375, 0.0501708984375], [0.0029296875, 1.0, 0.0, 0.0003662109375, 0.0003662109375, 0.0003662109375, 0.07177734375, 0.00830078125, 0.004150390625, 0.0628662109375, 0.0224609375, 0.00146484375, 0.001220703125, 0.3709716796875, 0.031005859375, 0.00732421875, 0.0059814453125, 0.026611328125, 0.02783203125, 0.080810546875], [0.0042724609375, 0.0, 1.0, 0.068603515625, 0.059814453125, 0.0474853515625, 0.0001220703125, 0.0040283203125, 0.00244140625, 0.0003662109375, 0.0, 0.0003662109375, 0.00048828125, 0.0, 0.0, 0.0001220703125, 0.0096435546875, 0.0, 0.00048828125, 0.0], [